In [11]:
import pandas as pd
import requests as rq
import json
from io import StringIO
from datetime import timedelta, date, datetime
import numpy as np
from sklearn import preprocessing
import plotly.graph_objects as go

# Import Data

In [3]:
with open('stocks_fundamentals-nyse.json') as f:
    new_stock_fundamentals = json.load(f)
    

In [42]:
stock_codes = []
for s in new_stock_fundamentals:
    stock_codes.append(s['General']['Code'])

#print(new_stock_fundamentals[0])
stock_codes.append('Date')


In [43]:

### Start New
stock_quotes = pd.DataFrame(columns=stock_codes)
stock_quotes = stock_quotes.set_index('Date')

### Get Quotes from existing ones
#stock_quotes = pd.read_csv('2016_stock_quotes-nyse.csv')
#print(new_stock_quotes)
#stock_quotes = stock_quotes.set_index('Date')

#print(stock_quotes.index)

In [44]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

start_date = date(2016, 7, 1)
end_date = date(2017, 4, 1)
for single_date in daterange(start_date, end_date):
    day = single_date.strftime("%Y-%m-%d")
    print(day)
    daily_quotes = rq.get('http://eodhistoricaldata.com/api/eod-bulk-last-day/US?api_token=&fmt=csv&date='+day)
    df = pd.read_csv(StringIO(daily_quotes.text))
    df = df.set_index('Code')
    #print(df)
    quotes = []
    for code in stock_quotes.columns:
        if code in df.index:
            quotes.append(df.loc[code]['Close'])
        else:
            quotes.append(None)
    stock_quotes.loc[day] = quotes
    
    

2016-07-01
2016-07-02
2016-07-03
2016-07-04
2016-07-05
2016-07-06
2016-07-07
2016-07-08
2016-07-09
2016-07-10
2016-07-11
2016-07-12
2016-07-13
2016-07-14
2016-07-15
2016-07-16
2016-07-17
2016-07-18
2016-07-19
2016-07-20
2016-07-21
2016-07-22
2016-07-23
2016-07-24
2016-07-25
2016-07-26
2016-07-27
2016-07-28
2016-07-29
2016-07-30
2016-07-31
2016-08-01
2016-08-02
2016-08-03
2016-08-04
2016-08-05
2016-08-06
2016-08-07
2016-08-08
2016-08-09
2016-08-10
2016-08-11
2016-08-12
2016-08-13
2016-08-14
2016-08-15
2016-08-16
2016-08-17
2016-08-18
2016-08-19
2016-08-20
2016-08-21
2016-08-22
2016-08-23
2016-08-24
2016-08-25
2016-08-26
2016-08-27
2016-08-28
2016-08-29
2016-08-30
2016-08-31
2016-09-01
2016-09-02
2016-09-03
2016-09-04
2016-09-05
2016-09-06
2016-09-07
2016-09-08
2016-09-09
2016-09-10
2016-09-11
2016-09-12
2016-09-13
2016-09-14
2016-09-15
2016-09-16
2016-09-17
2016-09-18
2016-09-19
2016-09-20
2016-09-21
2016-09-22
2016-09-23
2016-09-24
2016-09-25
2016-09-26
2016-09-27
2016-09-28
2016-09-29

In [46]:
stock_quotes.to_csv('2016_stock_quotes-nyse.csv')

  # Preprocess stocks data


In [53]:
new_stock_quotes = pd.read_csv('2016_stock_quotes-nyse.csv')
new_stock_quotes = new_stock_quotes.set_index('Date')
print(new_stock_quotes)

                A     AA     AAMC    AAN     AAP    AAT    AAU     AB    ABB  \
Date                                                                           
2016-07-01  44.60    NaN  14.7920  22.38  164.33  42.53  1.660  23.17  19.91   
2016-07-02    NaN    NaN      NaN    NaN     NaN    NaN    NaN    NaN    NaN   
2016-07-03    NaN    NaN      NaN    NaN     NaN    NaN    NaN    NaN    NaN   
2016-07-04    NaN    NaN      NaN    NaN     NaN    NaN    NaN    NaN    NaN   
2016-07-05  43.75    NaN  13.8500  22.17  161.19  43.28  1.500  22.59  19.31   
...           ...    ...      ...    ...     ...    ...    ...    ...    ...   
2017-03-27  53.24  32.47  61.5000  28.82  147.80  41.38  1.110  21.60  23.37   
2017-03-28  53.44  32.92  67.9500  29.04  148.36  41.29  1.070  21.95  23.44   
2017-03-29  53.12  32.91  68.5000  29.30  149.67  41.18  1.120  21.95  23.36   
2017-03-30  53.32  33.49  74.9999  29.85  148.67  41.57  1.065  22.50  23.34   
2017-03-31  52.87  34.40  75.2500  29.74

In [54]:
cols = new_stock_quotes.columns
for c in cols:
    quotes = new_stock_quotes[c]
    non_nan_count = np.count_nonzero(~np.isnan(quotes))
    if non_nan_count < 75:
        del new_stock_quotes[c]

In [55]:
new_stock_quotes = new_stock_quotes.fillna(method='ffill')


In [56]:
start_date = '2016-01-01 '
today = datetime.now().date().strftime("%Y-%m-%d")

new_stock_quotes = new_stock_quotes.loc[start_date:today]
normalized = new_stock_quotes.apply(lambda x: (x / x.iloc[1]))
print(normalized)
#print(normalized.loc['2020-03-05'])
#normalized = normalized.drop(['2020-03-05'])

                   A  AA      AAMC       AAN       AAP       AAT       AAU  \
Date                                                                         
2016-07-01  1.000000 NaN  1.000000  1.000000  1.000000  1.000000  1.000000   
2016-07-02  1.000000 NaN  1.000000  1.000000  1.000000  1.000000  1.000000   
2016-07-03  1.000000 NaN  1.000000  1.000000  1.000000  1.000000  1.000000   
2016-07-04  1.000000 NaN  1.000000  1.000000  1.000000  1.000000  1.000000   
2016-07-05  0.980942 NaN  0.936317  0.990617  0.980892  1.017635  0.903614   
...              ...  ..       ...       ...       ...       ...       ...   
2017-03-27  1.193722 NaN  4.157653  1.287757  0.899410  0.972960  0.668675   
2017-03-28  1.198206 NaN  4.593699  1.297587  0.902818  0.970844  0.644578   
2017-03-29  1.191031 NaN  4.630882  1.309205  0.910789  0.968258  0.674699   
2017-03-30  1.195516 NaN  5.070302  1.333780  0.904704  0.977428  0.641566   
2017-03-31  1.185426 NaN  5.087209  1.328865  0.902209  0.983776

# By Sector

In [57]:
by_sector = {}
for s in new_stock_fundamentals:
    sector = s["General"]["Sector"]
    if sector not in by_sector:
        by_sector[sector] = []
    by_sector[sector].append(s["General"]["Code"])

print(by_sector.keys())

dict_keys(['Healthcare', 'Basic Materials', 'Financial Services', 'Industrials', 'Consumer Cyclical', 'Real Estate', 'Consumer Defensive', 'Other', 'Technology', 'Communication Services', 'Energy', 'Utilities', '', None, 'Services', 'Financial', 'Industrial Goods', 'Consumer Goods'])


In [58]:
def avg_symbols_perf(symbols, data):
    df = pd.DataFrame(columns=symbols)
    df2 = data.filter(items=df.columns)#pd.merge(df,data, on=df.columns, how="inner")
    df2 = df2.mean(axis=1)
    df2 = df2.replace([np.inf, -np.inf], np.nan)
    return df2


In [59]:
returns_by_sector = pd.DataFrame(columns=by_sector.keys())
for k in by_sector:
    #print(by_sector[k])
    returns_by_sector[k] = avg_symbols_perf(by_sector[k],normalized)
    

returns_by_sector['Financial Services'] = None
returns_by_sector['Basic Materials'] = None

print(returns_by_sector)

            Healthcare Basic Materials Financial Services  Industrials  \
Date                                                                     
2016-07-01    1.000000            None               None     1.000000   
2016-07-02    1.000000            None               None     1.000000   
2016-07-03    1.000000            None               None     1.000000   
2016-07-04    1.000000            None               None     1.000000   
2016-07-05    0.985571            None               None     0.984878   
...                ...             ...                ...          ...   
2017-03-27    1.429359            None               None     1.811121   
2017-03-28    1.460660            None               None     1.840301   
2017-03-29    1.444977            None               None     1.862030   
2017-03-30    1.443761            None               None     1.864102   
2017-03-31    1.444436            None               None     1.885859   

            Consumer Cyclical  Real E

In [61]:
fig = go.Figure()
#print(returns_by_sector)
for k in returns_by_sector:
    print(k)
   # print(returns_by_sector[k])
    fig.add_trace(go.Scatter(
                    x=returns_by_sector.index,
                    y=returns_by_sector[k],
                    name=k,
                    #line_color='deepskyblue',
                    opacity=1))
fig.show()

Healthcare
Basic Materials
Financial Services
Industrials
Consumer Cyclical
Real Estate
Consumer Defensive
Other
Technology
Communication Services
Energy
Utilities

None
Services
Financial
Industrial Goods
Consumer Goods
